In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
os.getcwd()

'c:\\Users\\defco\\OneDrive\\Escritorio\\Cursos\\Programación\\Cursados\\Data Science Bootcamp\\2503_dsft_thebridge_anderpena\\2-Data_Analysis\\8-BBDD\\SQL\\Practica\\4-Murder_game'

In [3]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("data/sql-murder-mystery.db")

In [ ]:
# Buscamos los reports:

query = """
SELECT * FROM crime_scene_report AS C
WHERE date = 20180115 and city = 'SQL City'
"""

reports = pd.read_sql_query(query, connection)
reports

,date,type,description,city
0,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot hi...",SQL City
1,20180115,assault,Report Not Found,SQL City
2,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [7]:
for i in reports['description']:
    print(i)

Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!
Report Not Found
Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


In [ ]:
# Primer sospechoso:

query = """
SELECT * FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1
"""

pd.read_sql_query(query, connection)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [ ]:
# Obtenemos la entrevista al testigo Morty:

query = """
SELECT * FROM person
JOIN interview ON person.id = interview.person_id
WHERE person.id = 14887;
"""

interview = pd.read_sql_query(query, connection)
interview

,id,name,license_id,address_number,address_street_name,ssn,person_id,transcript
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949,14887,I heard a gunshot and then saw a man run out. ...


In [27]:
interview['transcript'][0]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

In [30]:
# Annabel en Franklin Ave


query = """
SELECT * FROM person
WHERE name LIKE 'Annabel%' AND address_street_name = 'Franklin Ave';
"""

pd.read_sql_query(query, connection)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [ ]:
# Obtenemos la entrevista a la testigo Annabel:

query = """
SELECT * FROM person
JOIN interview ON person.id = interview.person_id
WHERE person.id = 16371;
"""

interview = pd.read_sql_query(query, connection)
interview

,id,name,license_id,address_number,address_street_name,ssn,person_id,transcript
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143,16371,"I saw the murder happen, and I recognized the ..."


In [32]:
interview['transcript'][0]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

In [46]:
# Buscamos el sospechoso a través de los datos que dan en los interrogatorios:
# Martin: 'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'
# Annabel: 'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

query = """
SELECT * FROM get_fit_now_check_in as gym_check
JOIN get_fit_now_member as gym_member ON gym_check.membership_id = gym_member.id
JOIN person as p ON gym_member.person_id = p.id
JOIN drivers_license as dl ON p.license_id = dl.id
WHERE gym_check.membership_id LIKE '48Z%' 
    AND gym_check.check_in_date = 20180109
    AND gym_member.membership_status = 'gold'
    AND dl.plate_number LIKE '%H42W%';
"""

pd.read_sql_query(query, connection)

,membership_id,check_in_date,check_in_time,check_out_time,id,person_id,name,membership_start_date,membership_status,id,...,ssn,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,48Z55,20180109,1530,1700,48Z55,67318,Jeremy Bowers,20160101,gold,67318,...,871539279,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


In [ ]:
# Obtenemos la entrevista al sospechoso Jeremy Bowers:

query = """
SELECT * FROM person
JOIN interview ON person.id = interview.person_id
WHERE person.id = 67318;
"""

interview = pd.read_sql_query(query, connection)
interview

,id,name,license_id,address_number,address_street_name,ssn,person_id,transcript
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,67318,I was hired by a woman with a lot of money. I ...


In [ ]:
interview['transcript'][0]

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

ASESINO: JEREMY BOWERS

In [51]:
# Buscamos la sospechosa que indica Jeremy:

# 'I was hired by a woman with a lot of money. I don\'t know her name but I know 
# she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a
# Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in 
# December 2017.\n'

query = """
SELECT * FROM drivers_license as dl
JOIN person as p ON dl.id = p.license_id
JOIN facebook_event_checkin as fb ON p.id = fb.person_id
WHERE dl.height BETWEEN 65 AND 67
    AND dl.hair_color = 'red'
    AND dl.car_make = 'Tesla'
    AND dl.car_model = 'Model S'
    AND fb.event_name = 'SQL Symphony Concert'
    AND fb.date BETWEEN 20171201 AND 20171231;
"""

pd.read_sql_query(query, connection)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn,person_id,event_id,event_name,date
0,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,1143,SQL Symphony Concert,20171206
1,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,1143,SQL Symphony Concert,20171212
2,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,1143,SQL Symphony Concert,20171229


In [ ]:
# Obtenemos el interrogatorio a la sospechosa Miranda Priestly:

query = """
SELECT * FROM person
JOIN interview ON person.id = interview.person_id
WHERE person.id = 99716;
"""

interview = pd.read_sql_query(query, connection)
interview

,id,name,license_id,address_number,address_street_name,ssn,person_id,transcript


No hay interrogatorio.

In [55]:
# Obtenemos su income anual para ver si tiene muchos ingresos:

query = """
SELECT * FROM income
WHERE ssn = 987756388;
"""

pd.read_sql_query(query, connection)

,ssn,annual_income
0,987756388,310000


Es rica. Coincide con la descripción. Tenemos a nuestra cabeza pensante.

IDEADORA: MIRANDA PRIESTLEY